In [1]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(14, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [2]:
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    import ast
    authorList = ast.literal_eval(row.authorList)
    
    parameters = {
        # 'candidate': [2, 3, 4, 5],
        # 'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
        'samples': [50, 100, 200, 400, 800, 1600, 3200, 6400],
        # 'dimensions': [50, 100, 200],
        # 'iterations': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200],
    }
    
    print('doc_id: %3s actual author: %4s author_id: %s' % (str(doc_id), str(row.author_id), str(authorList)))
    
    level = 'word'
    
    for idxp, paralist in parameters.iteritems():
        if idxp == 'samples':
            for idxl, val in enumerate(paralist):
                doc_id = row.doc_id
                
                author_id = row.author_id

                import ast
                authorList = ast.literal_eval(row.authorList)

                candidate = len(authorList)

                test = idxp # change before run

                level = "word"

                iterations = 30

                dropout = 0.4

                samples = val

                loc = authorList.index(author_id)

                printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
                printstate += (("dimensions = %s, samples = %s, ") % (str(200), str(samples)))
                printstate += (("\niterations = %s, dropout = %s, ") % (str(iterations), str(dropout)))
                printstate += (("test = %s") % (str(test)))

                print("Current test: %s" % (str(printstate)))

                import UpdateDB as db

                case = db.checkOldCNN(doc_id = doc_id, candidate = val, dimensions = 200, samples = 300, 
                                      iterations = iterations, dropout = dropout, test = test)

                if case == False:

                    print("Running: %12s" % (str(printstate)))

                    import StyloNeural as Stylo
                    (labels_index, predYList, predY, history, samples) = Stylo.getResults(
                        doc_id = doc_id, authorList = authorList[:], 
                        level = level, glove = '../../glove/', dimensions = 200, 
                        samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

                    ans = predY[loc]
                    
                    import UpdateDB as db
                    case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = 200, 
                                                 samples = samples, iterations = iterations, dropout = dropout, 
                                                 accuracy = ans, test = test)

                    ans = 0.0

                    if (predY.tolist().index(max(predY)) == loc):
                        ans = 1.0
                        
                    
                    import UpdateDB as db
                    case = db.updateresultOldCNN(doc_id = doc_id, candidate = candidate, dimensions = 200, 
                                                 samples = samples, iterations = iterations, dropout = dropout, 
                                                 accuracy = ans, test = str(test + 'binary'))

                    from IPython.display import clear_output

                    clear_output()
                else:
                    print("Skipped: %12s test:   index: %2s" % (str(idxp), str(idxl)))
                

doc_id: 204 actual author:  123 author_id: (123, 81, 23, 214, 161)
Current test: iterations
  iterations test:   index:  0
Error (1146, "Table 'tests.readings' doesn't exist")


SystemExit: 1

/home/ninadt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb